# Linear equations

In [ ]:
import pandas as pd
import numpy as np

from algorithms_linear import gauss_seidel
from algorithms_linear import gauss_jacobi
from algorithms_linear import forward_substitution
from algorithms_linear import backward_substitution
from algorithms_linear import solve

from plots_linear import plot_operation_count
from plots_linear import plot_ill_problem_2
from plots_linear import plot_iterative_convergence

from problems_linear import get_ill_problem_1
from problems_linear import get_ill_problem_2
from problems_linear import get_inverse_demand_problem


import time

# how come using autformatting?
np.set_printoptions(precision=4)
# also for Pandas

## Special cases

We can start with some special cases to develop the basic building blocks for more complicated material

In [ ]:
A = np.identity(3)
b = np.random.normal(size=3)

x = forward_substitution(A, b)
x = backward_substitution(A, b)
x, b

## L-U Factorization







Adding to this the two building blocks we developed earlier `forward_substitution` and `backward_substitution`, we can now write a quite generic function to solve systems of linear equations.

In [ ]:
A = np.array([[3, 1], [1, 2]])
b = np.array([9, 8])

x = solve(A, b)
# Test that correct.
np.allclose(np.dot(A, x), b)

Building your own numerical routines is usually the only way to really understand the algorithms and learn about all the potential pitfalls. However, the default should be to rely on battle-tested production code. For linear algebra there are numerous well established libraries available.

How does solving a system of linear equations compare to other alternative?

In [ ]:
plot_operation_count()

The right setup for your numerical needs depends on your particular problem. For example, this trade-off looks very different if you have to solve numerous linear equations that only differ in $b$ but not $A$. 

In [ ]:
def tic():
    return time.time()


def toc(t):
    return time.time() - t


print(
    "{:^5} {:^5}   {:^11} {:^11} \n{}".format(
        "m", "n", "np.solve(A,b)", "dot(inv(A), b)", "-" * 40
    )
)

for m in [1, 100]:
    for n in [50, 500]:
        A = np.random.rand(n, n)
        b = np.random.rand(n, 1)

        tt = tic()
        for j in range(m):
            x = np.linalg.solve(A, b)

        f1 = 100 * toc(tt)

        tt = tic()
        Ainv = np.linalg.inv(A)
        for j in range(m):
            x = np.dot(Ainv, b)

        f2 = 100 * toc(tt)
        print(" {:3}   {:3} {:11.2f} {:11.2f}".format(m, n, f1, f2))

## Gaussian elimination

code in file, give reference and then move on.


## Rounding error

##  ill conditioned

In [ ]:
grid = [5, 10, 15]
cond = np.tile(np.nan, len(grid))
err = np.tile(np.nan, len(grid))
for i, n in enumerate(grid):
    A, b, x_true = get_ill_problem_1(n)
    x_solve = np.linalg.solve(A, b)
    cond[i] = np.linalg.cond(A)
    err[i] = np.linalg.norm(x_solve - x_true, 1)

In [ ]:
df = pd.DataFrame(columns=["Condition", "Error", "Dimension"])

df["Dimension"] = grid
df["Condition"] = err
df["Error"] = cond

In [ ]:
df

In [ ]:
grid = np.linspace(0.9, 1.1)
cond, err = list(), list()
for p in grid:
    A, b, x_true = get_ill_problem_2(p)
    x_solve = np.linalg.solve(A, b)

    cond.append(np.linalg.cond(A))
    err.append(np.linalg.norm(x_solve - x_true, 1))

In [ ]:
plot_ill_problem_2(cond, err, grid)

## Iterative methods


In [ ]:
A, b, x_true = get_inverse_demand_problem()

### Gauss Jacobi

In [ ]:
x_solve, conv_gj = gauss_jacobi(A, b)

In [ ]:
x_solve, x_true

### Gauss Seidel

In [ ]:
x_solve, conv_gs = gauss_seidel(A, b)

We can now compare the two methods.

In [ ]:
plot_iterative_convergence(conv_gs, conv_gj)

----